<a href="https://colab.research.google.com/github/snassimr/ApplicationsBuildWithLLMs/blob/main/deoptima_generation_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [122]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [123]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [124]:
SYS_PROJECT_DIR  = '/content/gdrive/MyDrive/Colab Notebooks/deoptima'
SYS_SRC_DIR  = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/deoptima'
SYS_INPUT_DIR    = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/input'
SYS_MODELING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/modeling'
SYS_TESTING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/testing'
SYS_RUNTIME_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/runtime'
SYS_LLM_MODEL    = 'Mistral-7B-Instruct-v0.3'

In [125]:
import os
import pandas as pd
import shutil

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.copy_on_write = True

# Installs

In [126]:
import os

requirements = """
torch==2.3.1
accelerate==0.32.1
transformers==4.42.4
vllm==0.5.4
gradio==4.41.0
nemoguardrails==0.9.1.1
langchain_community==0.2.11
langchain-openai==0.1.21
"""
requirements = """
python-dotenv==1.0.1
gradio==4.41.0
nemoguardrails==0.9.1.1
langchain-openai==0.1.21
"""

requirements_path = os.path.join(SYS_PROJECT_DIR, 'requirements.txt')
with open(requirements_path, 'w') as f:
    f.write(requirements)

86

In [127]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [128]:
import os
requirements_path = os.path.join(SYS_PROJECT_DIR, 'requirements.txt')
!pip install -q -r '{requirements_path}'

In [129]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

from deoptima.utils import set_logger
from deoptima.utils import dictionary_to_string
from deoptima.utils import conversation_to_string
from deoptima.utils import process_state_to_string

In [130]:
from deoptima.utils import generate_session_id
session_id = generate_session_id()

In [131]:
from deoptima.utils import set_logger
deoptima_logger = set_logger(session_id, log_mode = 'FS')

In [132]:
import logging
deoptima_logger = logging.getLogger("deoptima")

In [133]:
# import os

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# openai_api_key = userdata.get("OPENAI_API_KEY")

In [134]:
from dotenv import load_dotenv, find_dotenv
# Get the current environment, default to 'dev'
ENV = os.getenv('ENV', 'dev')

# Load the appropriate .env file based on the ENV variable
if ENV == 'prod':
  _ = load_dotenv(os.path.join(SYS_PROJECT_DIR,'./.env.prod'))
else:
  _ = load_dotenv(os.path.join(SYS_PROJECT_DIR,'./.env.dev'))

# Model

## Conversation

In [135]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

from deoptima.utils import load_from_json

In [136]:
conversation_input_folder = os.path.join(SYS_TESTING_DIR, "input")

# conversation_file = "process_1.json
# conversation_file = "process_4.json"
# conversation_file = "start_process_1.json"
# conversation_file = "start_process_2.json"
# conversation_file = "assistant_stop_1.json"
# conversation_file = "assistant_stop_2.json"
# conversation_file = "process_test_options_processstate.json"
conversation_file = "recent_1.json"

conversation = load_from_json(os.path.join(conversation_input_folder, conversation_file))

from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

## get_process_state

In [146]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

from deoptima.utils import conversation_to_string

In [147]:
import os
import yaml

with open(os.path.join(SYS_SRC_DIR, 'generation_data.yaml'), 'r') as file:
    generation_data = yaml.safe_load(file)

In [139]:
def format_criteria_comparisions_list(actual_list, criteria_list):

    from itertools import combinations

    all_combinations = list(combinations(criteria_list, 2))

    actual_comparisons = {
        (comp['criterion_1'], comp['criterion_2']): comp['criteria_rating']
        for comp in actual_list
    }

    filled_list = []

    for criterion_1, criterion_2 in all_combinations:
        comparison = {
            'criterion_1': criterion_1,
            'criterion_2': criterion_2,
            'criteria_rating': 'Missing'
        }
        if (criterion_1, criterion_2) in actual_comparisons:
            comparison['criteria_rating'] = actual_comparisons[(criterion_1, criterion_2)]
        filled_list.append(comparison)

    return filled_list

def format_preferences_dict1(actual_dict, options_list, criteria_list):
    import itertools

    filled_dict = []

    for criterion in criteria_list:
        # Find the corresponding actual entry for this criterion, if it exists
        actual_entry = next((item for item in actual_dict if item['criterion'] == criterion), None)

        filled_preferences = []
        for option_1, option_2 in itertools.combinations(options_list, 2):
            # Check if actual entry exists for this specific option pair
            if actual_entry:
                matching_actual_pref = next(
                    (pref for pref in actual_entry['preference_statements']
                     if pref['option_1'] == option_1 and pref['option_2'] == option_2),
                    None
                )
            else:
                matching_actual_pref = None

            # If actual preference exists, use it; otherwise, fill with "Missing"
            if matching_actual_pref:
                filled_preferences.append(matching_actual_pref)
            else:
                filled_preferences.append({
                    'option_1': option_1,
                    'option_2': option_2,
                    'preference_rating': 'Missing'
                })

        # Add the filled preferences for the current criterion
        filled_dict.append({
            'criterion': criterion,
            'preference_statements': filled_preferences
        })

    return filled_dict

def format_preferences_dict2(actual_dict, options_list, criteria_list):
    import itertools
    filled_dict = {}

    for criterion in criteria_list:
        # Get the preference statements for this criterion
        actual_preferences = actual_dict.get(criterion, [])

        # Create a list to hold the filled preferences for this criterion
        criterion_preferences_list = []

        for option_1, option_2 in itertools.combinations(options_list, 2):
            # Check if there is a matching preference statement for this option pair
            matching_preference = next(
                (pref['preference_rating'] for pref in actual_preferences
                 if pref['option_1'] == option_1 and pref['option_2'] == option_2),
                'Missing'  # Default to 'Missing' if no match is found
            )

            # Create a dictionary entry for this option pair
            preference_entry = {
                'option_1': option_1,
                'option_2': option_2,
                'preference_rating': matching_preference
            }
            criterion_preferences_list.append(preference_entry)

        # Add the filled preferences for the current criterion to filled_dict
        filled_dict[criterion] = criterion_preferences_list

    return filled_dict


In [140]:
criteria_rating_list = generation_data['infos']['criteria_rating_list']
preference_rating_list = generation_data['infos']['preference_rating_list']
process_technical_explanation = generation_data['infos']['process_technical_explanation']
process_technical_explanation = process_technical_explanation.format(
    criteria_rating_list=criteria_rating_list,
    preference_rating_list=preference_rating_list)
get_process_state_prompt = generation_data['prompts']['get_process_state']['prompt']
get_process_state_prompt = get_process_state_prompt.format(process_technical_explanation=process_technical_explanation)

In [141]:
from IPython.display import Markdown
display(Markdown(f"{get_process_state_prompt}"))


Process Technical Explanation
=============================

Step 1 - Basic Definitions: User should provide following list of definitions during conversation with Deoptima:

-	Step 1.1 Goal the user trying to achieve in optimal way
  Examples:
    -	selecting the best laptop
    -	selecting the best destination for your vacation
-	Step 1.2 Options list: options (or alternatives) are the different choices or possibilities that the user can select from to achieve goal
  Examples:
    -	Laptop model 1, Laptop model 2, Laptop model 3
    -	Vacation Destination 1, Vacation Destination 2, Vacation Destination 3
  Notation:
    -	n_o – number of options / options list length
-	Step 1.3 Criteria list: Criteria are the important factors that will influence your decision. Think of them as the things user care most about when choosing the best option.
  Examples:
    -	Price, Performance, Battery Life
    -	Price, Climate, Activities
  Notation:
    -	n_c – number of criteria / criteria list length

Step 2 - basic definitions are set and next step is providing criteria comparision list

Step 3 - Criteria Comparison List

User should assign a criteria rating to each valid criteria pair.
Valid criteria pairs are combinations of pairs from criteria list without considering the order of importance within each pair
Example:
For criteria list = ['Price', 'Performance', 'Climate'], the valid criteria pairs are:
- criterion_1 = "Price", criterion_2 = "Performance"
- criterion_1 = "Price", criterion_2 = "Climate"
- criterion_1 = "Performance", criterion_2 = "Climate"

During the each comparison user should assign a rating come from following criteria rating list to a criteria pair:

['Equally Important', 'Moderately More Important', 'Strongly More Important', 'Very Strongly More Important', 'Extremely More Important', 'Moderately Less Important', 'Strongly Less Important', 'Very Strongly Less Important', 'Extremely Less Important']

The valid criteria comparison statement format:
[criterion_1]  is [criteria_rating]  to/than [criterion_2] , where : criterion_1 and criterion_2 is a valid criteria pair and criteria_rating is value comes from criteria_rating_list
Examples: For criteria_list = ['Price', 'Performance', 'Climate'], sample criteria comparision list is:
- criterion_1 = "Price", criterion_2 = "Performance", criteria_rating = "Equally Important" -> "'Price' is Equally Important to 'Performance'"
- criterion_1 = "Price", criterion_2 = "Climate", criteria_rating = "Moderately More Important" -> "'Price' is Moderately More Important than 'Performance'"
- criterion_1 = "Performance", criterion_2 = "Climate", criteria_rating = "Extremely Less Important" -> "'Price' is Extremely Less Important than 'Performance'"

Notation:
  -	n_cc – number of criteria comparison statements / criteria comparision list length

Step 4 - criteria comparisions are set and and next step is providing preferences statements.

Step 5 - Preferences Dictionary

Preferences dictionary contains preferences list for each criterion in criteria list.

Preferences list contains a list of valid preferences statements for specific criterion.

Valid options pairs are combinations of pairs from options list without considering the order of importance within each pair.
The order between options pairs is important.
Example: For options list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], valid options pairs are:
- option_1 = "Laptop 1", option_2 = "Laptop 2"
- option_1 = "Laptop 1", option_2 = "Laptop 3"
- option_1 = "Laptop 2", option_2 = "Laptop 3"

The valid preference statement format:
[criterion]: [option_1] is [preference_rating] to [option_2], 
where criterion is a value from criteria list, option_1 and option_2 is a valid options pair, preferences_rating is value comes from preference_rating_list:

['Equally Preferred', 'Moderately More Preferred', 'Strongly More Preferred', 'Very Strongly More Preferred', 'Extremely More Preferred', 'Moderately Less Preferred', 'Strongly Less Preferred', 'Very Strongly Less Preferred', 'Extremely Less Preferred']

Preferences dictionary contains criteria in the order as they appear in the criteria list, while the preference statements follow the order of the valid options pairs.
Example: For criteria_list = ['Price', 'Performance', 'Climate'] and options_list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], sample preferences dictionary is:
-	criterion = "Price", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Equally Preferred" -> preference_statement = "'Price': 'Laptop 1' is Equally Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Price': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly Less Preferred" -> preference_statement = "'Price': 'Laptop 2' is Very Strongly Less Preferred than 'Laptop 3'"
-	criterion = "Performance", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Extremely More Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Extremely More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately Less Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Moderately Less Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Performance': 'Laptop 2' is Very Strongly More Preferred than 'Laptop 3'"
-	criterion = "Climate", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Very Strongly More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Extremely Less Preferred" -> preference_statement = "'Climate': 'Laptop 2' is Extremely Less Preferred than 'Laptop 3'"

Notation:
  -	n_oc – for givern criterion , number of preference statements / preferences list length

Step 6 - all definitions are set and the solution will be prepared.


Task
============

Analyze input conversation between Deoptima assistant and user. During the conversation each process state component can be updated.
Your goal is to extract the most recent values for goal, options list, criteria list, criteria comparison list and preferences dictionary in this order.

While extracting criteria comparison list, carefully follow these steps:
  - For each criteria pair set criteria_rating to latest value as stated in conversation. Provide a detailed step-by-step explanation to get latest value For each criteria pair.
  - Output criteria comparison list.  

While extracting preferences dictionary, carefully follow these steps:
  - For each criterion and options pair set preferences_rating to latest value as stated in conversation.
  - Output preferences dictionary.


In [142]:
from typing import Optional, List, Dict, Tuple

def get_process_state(prompt : str, conversation: str):
  import os
  from pydantic import BaseModel, Field
  from langchain_core.utils.function_calling import convert_to_openai_function
  from langchain_openai import ChatOpenAI
  from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
  from langchain_core.prompts import ChatPromptTemplate

  model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.0)

  class Criteria_Comparision_Statement(BaseModel):
    """Criteria Comparision Statement."""
    criterion_1: str  = Field(description="First criterion")
    criterion_2: str  = Field(description="Second criterion")
    criteria_rating: str = Field(description="Criteria rating")
  class Preference_Statement(BaseModel):
    """Preference Statement."""
    option_1: str  = Field(description="First option")
    option_2: str  = Field(description="Second option")
    preference_rating: str = Field(description="Preference rating")
  class Criterion_Preference_Statement(BaseModel):
    """Criterion Preference Statement."""
    criterion: str  = Field(description="Criterion")
    preference_statements: List[Preference_Statement] = Field(description="Preference statements.")
  class Process_State(BaseModel):
    """Deoptima process state."""
    goal_name: str = Field(description="Goal to achieve")
    options_list: List[str] = Field(description="Options list")
    criteria_list:List[str] = Field(description="Criteria list")
    criteria_comparison_list:List[Criteria_Comparision_Statement] = Field(description="List of criteria comparision statements. Contains the most recent criteria_rating for reach criteria pair.")
    preferences_dict: List[Criterion_Preference_Statement] = Field(description="Preferences dictionary.")

  tagging_prompt = ChatPromptTemplate.from_messages([
        ("system", prompt),
        ("human", "{input}"),
    ])

  structured_llm = model.with_structured_output(Process_State)

  chain = tagging_prompt | structured_llm

  response = chain.invoke({"input": conversation})

  response_dict = response.dict()

  print(response_dict['criteria_comparison_list'])

  response_dict['criteria_comparison_list'] = format_criteria_comparisions_list(response_dict['criteria_comparison_list'],
                                                                       response_dict['criteria_list'])
  preferences_dict = response_dict['preferences_dict']
  response_dict['preferences_dict'] = {item['criterion']: item['preference_statements'] for item in preferences_dict}

  response_dict['preferences_dict'] = format_preferences_dict2(response_dict['preferences_dict'],
                                                               response_dict['options_list'],
                                                               response_dict['criteria_list'])

  return response_dict

## get_process_state - New

In [155]:
criteria_rating_list = generation_data['infos']['criteria_rating_list']
preference_rating_list = generation_data['infos']['preference_rating_list']
process_technical_explanation = generation_data['infos']['process_technical_explanation']
process_technical_explanation = process_technical_explanation.format(
    criteria_rating_list=criteria_rating_list,
    preference_rating_list=preference_rating_list)
get_process_state_prompt = generation_data['prompts']['get_process_state']['prompt_conversation']
get_process_state_prompt = get_process_state_prompt.format(process_technical_explanation=process_technical_explanation,
                                                           conversation = "{conversation}")

In [156]:
  from IPython.display import Markdown
  display(Markdown(f"{get_process_state_prompt}"))


Process Technical Explanation
=============================

Step 1 - Basic Definitions: User should provide following list of definitions during conversation with Deoptima:

-	Step 1.1 Goal the user trying to achieve in optimal way
  Examples:
    -	selecting the best laptop
    -	selecting the best destination for your vacation
-	Step 1.2 Options list: options (or alternatives) are the different choices or possibilities that the user can select from to achieve goal
  Examples:
    -	Laptop model 1, Laptop model 2, Laptop model 3
    -	Vacation Destination 1, Vacation Destination 2, Vacation Destination 3
  Notation:
    -	n_o – number of options / options list length
-	Step 1.3 Criteria list: Criteria are the important factors that will influence your decision. Think of them as the things user care most about when choosing the best option.
  Examples:
    -	Price, Performance, Battery Life
    -	Price, Climate, Activities
  Notation:
    -	n_c – number of criteria / criteria list length

Step 2 - basic definitions are set and next step is providing criteria comparision list

Step 3 - Criteria Comparison List

User should assign a criteria rating to each valid criteria pair.
Valid criteria pairs are combinations of pairs from criteria list without considering the order of importance within each pair
Example:
For criteria list = ['Price', 'Performance', 'Climate'], the valid criteria pairs are:
- criterion_1 = "Price", criterion_2 = "Performance"
- criterion_1 = "Price", criterion_2 = "Climate"
- criterion_1 = "Performance", criterion_2 = "Climate"

During the each comparison user should assign a rating come from following criteria rating list to a criteria pair:

['Equally Important', 'Moderately More Important', 'Strongly More Important', 'Very Strongly More Important', 'Extremely More Important', 'Moderately Less Important', 'Strongly Less Important', 'Very Strongly Less Important', 'Extremely Less Important']

The valid criteria comparison statement format:
[criterion_1]  is [criteria_rating]  to/than [criterion_2] , where : criterion_1 and criterion_2 is a valid criteria pair and criteria_rating is value comes from criteria_rating_list
Examples: For criteria_list = ['Price', 'Performance', 'Climate'], sample criteria comparision list is:
- criterion_1 = "Price", criterion_2 = "Performance", criteria_rating = "Equally Important" -> "'Price' is Equally Important to 'Performance'"
- criterion_1 = "Price", criterion_2 = "Climate", criteria_rating = "Moderately More Important" -> "'Price' is Moderately More Important than 'Performance'"
- criterion_1 = "Performance", criterion_2 = "Climate", criteria_rating = "Extremely Less Important" -> "'Price' is Extremely Less Important than 'Performance'"

Notation:
  -	n_cc – number of criteria comparison statements / criteria comparision list length

Step 4 - criteria comparisions are set and and next step is providing preferences statements.

Step 5 - Preferences Dictionary

Preferences dictionary contains preferences list for each criterion in criteria list.

Preferences list contains a list of valid preferences statements for specific criterion.

Valid options pairs are combinations of pairs from options list without considering the order of importance within each pair.
The order between options pairs is important.
Example: For options list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], valid options pairs are:
- option_1 = "Laptop 1", option_2 = "Laptop 2"
- option_1 = "Laptop 1", option_2 = "Laptop 3"
- option_1 = "Laptop 2", option_2 = "Laptop 3"

The valid preference statement format:
[criterion]: [option_1] is [preference_rating] to [option_2], 
where criterion is a value from criteria list, option_1 and option_2 is a valid options pair, preferences_rating is value comes from preference_rating_list:

['Equally Preferred', 'Moderately More Preferred', 'Strongly More Preferred', 'Very Strongly More Preferred', 'Extremely More Preferred', 'Moderately Less Preferred', 'Strongly Less Preferred', 'Very Strongly Less Preferred', 'Extremely Less Preferred']

Preferences dictionary contains criteria in the order as they appear in the criteria list, while the preference statements follow the order of the valid options pairs.
Example: For criteria_list = ['Price', 'Performance', 'Climate'] and options_list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], sample preferences dictionary is:
-	criterion = "Price", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Equally Preferred" -> preference_statement = "'Price': 'Laptop 1' is Equally Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Price': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly Less Preferred" -> preference_statement = "'Price': 'Laptop 2' is Very Strongly Less Preferred than 'Laptop 3'"
-	criterion = "Performance", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Extremely More Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Extremely More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately Less Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Moderately Less Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Performance': 'Laptop 2' is Very Strongly More Preferred than 'Laptop 3'"
-	criterion = "Climate", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Very Strongly More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Extremely Less Preferred" -> preference_statement = "'Climate': 'Laptop 2' is Extremely Less Preferred than 'Laptop 3'"

Notation:
  -	n_oc – for givern criterion , number of preference statements / preferences list length

Step 6 - all definitions are set and the solution will be prepared.


Conversation
============

{conversation}

Task
============

Analyze input conversation between Deoptima assistant and user. During the conversation each process state component can be updated.
Your goal is to extract the most recent values for goal, options list, criteria list, criteria comparison list and preferences dictionary in this order.

While extracting criteria comparison list, carefully follow these steps:
  - For each criteria pair set criteria_rating to latest value as stated in conversation. Provide a detailed step-by-step explanation to get latest value For each criteria pair.
  - Output criteria comparison list.  

While extracting preferences dictionary, carefully follow these steps:
  - For each criterion and options pair set preferences_rating to latest value as stated in conversation.
  - Output preferences dictionary.


In [165]:
from typing import Optional, List, Dict, Tuple

def get_process_state(prompt : str, conversation: str):
  import os
  from pydantic import BaseModel, Field
  from langchain_core.utils.function_calling import convert_to_openai_function
  from langchain_openai import ChatOpenAI
  from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
  from langchain_core.prompts import ChatPromptTemplate

  model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.5)

  class Criteria_Comparision_Statement(BaseModel):
    """Criteria Comparision Statement."""
    criterion_1: str  = Field(description="First criterion")
    criterion_2: str  = Field(description="Second criterion")
    criteria_rating: str = Field(description="Criteria rating")
  class Preference_Statement(BaseModel):
    """Preference Statement."""
    option_1: str  = Field(description="First option")
    option_2: str  = Field(description="Second option")
    preference_rating: str = Field(description="Preference rating")
  class Criterion_Preference_Statement(BaseModel):
    """Criterion Preference Statement."""
    criterion: str  = Field(description="Criterion")
    preference_statements: List[Preference_Statement] = Field(description="Preference statements.")
  class Process_State(BaseModel):
    """Deoptima process state."""
    goal_name: str = Field(description="Goal to achieve")
    options_list: List[str] = Field(description="Options list")
    criteria_list:List[str] = Field(description="Criteria list")
    criteria_comparison_list:List[Criteria_Comparision_Statement] = Field(description="List of criteria comparision statements. Contains the most recent criteria_rating for reach criteria pair.")
    preferences_dict: List[Criterion_Preference_Statement] = Field(description="Preferences dictionary.")

  tagging_prompt = ChatPromptTemplate.from_messages([
        ("system", prompt),
        ("human", "{input}"),
    ])

  structured_llm = model.with_structured_output(Process_State)

  prompt = prompt.format(conversation = conversation)

  response = structured_llm.invoke(prompt)

  response_dict = response.dict()

  # print(response_dict['criteria_comparison_list'])

  # response_dict['criteria_comparison_list'] = format_criteria_comparisions_list(response_dict['criteria_comparison_list'],
  #                                                                      response_dict['criteria_list'])
  # preferences_dict = response_dict['preferences_dict']
  # response_dict['preferences_dict'] = {item['criterion']: item['preference_statements'] for item in preferences_dict}

  # response_dict['preferences_dict'] = format_preferences_dict2(response_dict['preferences_dict'],
  #                                                              response_dict['options_list'],
  #                                                              response_dict['criteria_list'])

  return response_dict, prompt

In [166]:
process_state, prompt = get_process_state(get_process_state_prompt, conversation_to_string(conversation))
process_state

{'goal_name': 'Selecting the best laptop',
 'options_list': ['Gaming Laptop', 'Ultrabook', '2-in-1 Laptop'],
 'criteria_list': ['Storage Capacity', 'Graphics Card', 'Portability'],
 'criteria_comparison_list': [{'criterion_1': 'Storage Capacity',
   'criterion_2': 'Graphics Card',
   'criteria_rating': 'Strongly Less Important'},
  {'criterion_1': 'Storage Capacity',
   'criterion_2': 'Portability',
   'criteria_rating': 'Extremely Less Important'},
  {'criterion_1': 'Graphics Card',
   'criterion_2': 'Portability',
   'criteria_rating': 'Strongly Less Important'}],
 'preferences_dict': [{'criterion': 'Storage Capacity',
   'preference_statements': [{'option_1': 'Gaming Laptop',
     'option_2': 'Ultrabook',
     'preference_rating': 'Equally Preferred'},
    {'option_1': 'Gaming Laptop',
     'option_2': '2-in-1 Laptop',
     'preference_rating': 'Strongly Less Preferred'},
    {'option_1': 'Ultrabook',
     'option_2': '2-in-1 Laptop',
     'preference_rating': 'Equally Preferred'}]}

In [163]:
print(prompt)


Process Technical Explanation

Step 1 - Basic Definitions: User should provide following list of definitions during conversation with Deoptima:

-	Step 1.1 Goal the user trying to achieve in optimal way
  Examples:
    -	selecting the best laptop
    -	selecting the best destination for your vacation
-	Step 1.2 Options list: options (or alternatives) are the different choices or possibilities that the user can select from to achieve goal
  Examples:
    -	Laptop model 1, Laptop model 2, Laptop model 3
    -	Vacation Destination 1, Vacation Destination 2, Vacation Destination 3
  Notation:
    -	n_o – number of options / options list length
-	Step 1.3 Criteria list: Criteria are the important factors that will influence your decision. Think of them as the things user care most about when choosing the best option.
  Examples:
    -	Price, Performance, Battery Life
    -	Price, Climate, Activities
  Notation:
    -	n_c – number of criteria / criteria list length

Step 2 - basic definitio

In [154]:
print(conversation_to_string(conversation))

user: I'm ready to start the decision-making process.
assistant: Great! Let's start by defining your goal. What is the goal you are trying to achieve in an optimal way?
user: Selecting the best laptop
assistant: Now, please provide a list of options you are considering for your laptop selection. What are the different laptop models you have in mind?
user: Gaming Laptop, Ultrabook, 2-in-1 Laptop
assistant: Now, let's define the criteria that will influence your decision in selecting the best laptop. What are the important factors you care about when choosing a laptop?
user: Storage Capacity, Graphics Card, Portability
assistant: Let's move on to the criteria comparison list. Please provide a comparison rating for the following criteria pair: 'Storage Capacity' is compared to 'Graphics Card'. How would you rate their importance?
user: 'Storage Capacity' is Strongly Less Important than 'Graphics Card'
assistant: Let's continue with the criteria comparison list. Please provide a comparison

In [151]:
process_state

{'goal_name': 'Selecting the best laptop',
 'options_list': ['Gaming Laptop', 'Ultrabook', '2-in-1 Laptop'],
 'criteria_list': ['Storage Capacity', 'Graphics Card', 'Portability'],
 'criteria_comparison_list': [{'criterion_1': 'Storage Capacity',
   'criterion_2': 'Graphics Card',
   'criteria_rating': 'Strongly Less Important'},
  {'criterion_1': 'Storage Capacity',
   'criterion_2': 'Portability',
   'criteria_rating': 'Extremely Less Important'},
  {'criterion_1': 'Graphics Card',
   'criterion_2': 'Portability',
   'criteria_rating': 'Strongly Less Important'}],
 'preferences_dict': [{'criterion': 'Storage Capacity',
   'preference_statements': [{'option_1': 'Gaming Laptop',
     'option_2': 'Ultrabook',
     'preference_rating': 'Equally Preferred'},
    {'option_1': 'Gaming Laptop',
     'option_2': '2-in-1 Laptop',
     'preference_rating': 'Strongly Less Preferred'},
    {'option_1': 'Ultrabook',
     'option_2': '2-in-1 Laptop',
     'preference_rating': 'Equally Preferred'}]}

In [24]:
import time
start_time = time.time()

try:
  process_state = get_process_state(get_process_state_prompt, conversation_to_string(conversation))
except Exception as e:
  deoptima_logger.error(f"Error in {get_process_state.__name__}: {e}", exc_info=True)
else:
  deoptima_logger.info(f"{get_process_state.__name__} finished: {str(process_state)}")

end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")

from IPython.display import Markdown
display(Markdown(f"{process_state}"))

deoptima - INFO     : 2024-10-27 00:10:29,598 : get_process_state finished: {'goal_name': 'Selecting the best laptop', 'options_list': ['Gaming Laptop', 'Ultrabook', '2-in-1 Laptop'], 'criteria_list': ['Storage Capacity', 'Graphics Card', 'Portability'], 'criteria_comparison_list': [{'criterion_1': 'Storage Capacity', 'criterion_2': 'Graphics Card', 'criteria_rating': 'Strongly Less Important'}, {'criterion_1': 'Storage Capacity', 'criterion_2': 'Portability', 'criteria_rating': 'Extremely Less Important'}, {'criterion_1': 'Graphics Card', 'criterion_2': 'Portability', 'criteria_rating': 'Strongly Less Important'}], 'preferences_dict': {'Storage Capacity': [{'option_1': 'Gaming Laptop', 'option_2': 'Ultrabook', 'preference_rating': 'Equally Preferred'}, {'option_1': 'Gaming Laptop', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Strongly Less Preferred'}, {'option_1': 'Ultrabook', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Equally Preferred'}], 'Graphics Card': [{'option_1'

Function runtime: 5747.88 msec


{'goal_name': 'Selecting the best laptop', 'options_list': ['Gaming Laptop', 'Ultrabook', '2-in-1 Laptop'], 'criteria_list': ['Storage Capacity', 'Graphics Card', 'Portability'], 'criteria_comparison_list': [{'criterion_1': 'Storage Capacity', 'criterion_2': 'Graphics Card', 'criteria_rating': 'Strongly Less Important'}, {'criterion_1': 'Storage Capacity', 'criterion_2': 'Portability', 'criteria_rating': 'Extremely Less Important'}, {'criterion_1': 'Graphics Card', 'criterion_2': 'Portability', 'criteria_rating': 'Strongly Less Important'}], 'preferences_dict': {'Storage Capacity': [{'option_1': 'Gaming Laptop', 'option_2': 'Ultrabook', 'preference_rating': 'Equally Preferred'}, {'option_1': 'Gaming Laptop', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Strongly Less Preferred'}, {'option_1': 'Ultrabook', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Equally Preferred'}], 'Graphics Card': [{'option_1': 'Gaming Laptop', 'option_2': 'Ultrabook', 'preference_rating': 'Equally Preferred'}, {'option_1': 'Gaming Laptop', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Equally Preferred'}, {'option_1': 'Ultrabook', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Strongly More Preferred'}], 'Portability': [{'option_1': 'Gaming Laptop', 'option_2': 'Ultrabook', 'preference_rating': 'Extremely More Preferred'}, {'option_1': 'Gaming Laptop', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Extremely More Preferred'}, {'option_1': 'Ultrabook', 'option_2': '2-in-1 Laptop', 'preference_rating': 'Extremely More Preferred'}]}}

In [25]:
from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

## get_response_data

In [26]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

from deoptima.utils import conversation_to_string
from deoptima.utils import process_state_to_string

In [27]:
import os
import yaml

with open(os.path.join(SYS_SRC_DIR, 'generation_data.yaml'), 'r') as file:
    generation_data = yaml.safe_load(file)

In [28]:
process_state

{'goal_name': 'Selecting the best laptop',
 'options_list': ['Gaming Laptop', 'Ultrabook', '2-in-1 Laptop'],
 'criteria_list': ['Storage Capacity', 'Graphics Card', 'Portability'],
 'criteria_comparison_list': [{'criterion_1': 'Storage Capacity',
   'criterion_2': 'Graphics Card',
   'criteria_rating': 'Strongly Less Important'},
  {'criterion_1': 'Storage Capacity',
   'criterion_2': 'Portability',
   'criteria_rating': 'Extremely Less Important'},
  {'criterion_1': 'Graphics Card',
   'criterion_2': 'Portability',
   'criteria_rating': 'Strongly Less Important'}],
 'preferences_dict': {'Storage Capacity': [{'option_1': 'Gaming Laptop',
    'option_2': 'Ultrabook',
    'preference_rating': 'Equally Preferred'},
   {'option_1': 'Gaming Laptop',
    'option_2': '2-in-1 Laptop',
    'preference_rating': 'Strongly Less Preferred'},
   {'option_1': 'Ultrabook',
    'option_2': '2-in-1 Laptop',
    'preference_rating': 'Equally Preferred'}],
  'Graphics Card': [{'option_1': 'Gaming Laptop',

In [29]:
process_state_to_string(process_state)

'---  ---\ngoal_name:\n\tSelecting the best laptop\noptions_list:\n\t- Gaming Laptop\n\t- Ultrabook\n\t- 2-in-1 Laptop\ncriteria_list:\n\t- Storage Capacity\n\t- Graphics Card\n\t- Portability\ncriteria_comparison_list:\n\t- criterion_1: Storage Capacity, criterion_2: Graphics Card, criteria_rating: Strongly Less Important\n\t- criterion_1: Storage Capacity, criterion_2: Portability, criteria_rating: Extremely Less Important\n\t- criterion_1: Graphics Card, criterion_2: Portability, criteria_rating: Strongly Less Important\npreferences_dict:\n\t- Storage Capacity:\n\t\t-- option_1: Gaming Laptop, option_2: Ultrabook, preference_rating: Equally Preferred\n\t\t-- option_1: Gaming Laptop, option_2: 2-in-1 Laptop, preference_rating: Strongly Less Preferred\n\t\t-- option_1: Ultrabook, option_2: 2-in-1 Laptop, preference_rating: Equally Preferred\n\t- Graphics Card:\n\t\t-- option_1: Gaming Laptop, option_2: Ultrabook, preference_rating: Equally Preferred\n\t\t-- option_1: Gaming Laptop, op

In [30]:
criteria_rating_list = generation_data['infos']['criteria_rating_list']
preference_rating_list = generation_data['infos']['preference_rating_list']
process_technical_explanation = generation_data['infos']['process_technical_explanation']
process_technical_explanation = process_technical_explanation.format(
    criteria_rating_list=criteria_rating_list,
    preference_rating_list=preference_rating_list)
get_response_data_prompt = generation_data['prompts']['get_response_data']['prompt']
get_response_data_prompt = get_response_data_prompt.format(process_technical_explanation=process_technical_explanation,
                                                           conversation = conversation_to_string(conversation),
                                                           process_state=process_state_to_string(process_state))

In [31]:
from IPython.display import Markdown
display(Markdown(f"{get_response_data_prompt}"))

Process Technical Explanation
=============================

Step 1 - Basic Definitions: User should provide following list of definitions during conversation with Deoptima:

-	Step 1.1 Goal the user trying to achieve in optimal way
  Examples:
    -	selecting the best laptop
    -	selecting the best destination for your vacation
-	Step 1.2 Options list: options (or alternatives) are the different choices or possibilities that the user can select from to achieve goal
  Examples:
    -	Laptop model 1, Laptop model 2, Laptop model 3
    -	Vacation Destination 1, Vacation Destination 2, Vacation Destination 3
  Notation:
    -	n_o – number of options / options list length
-	Step 1.3 Criteria list: Criteria are the important factors that will influence your decision. Think of them as the things user care most about when choosing the best option.
  Examples:
    -	Price, Performance, Battery Life
    -	Price, Climate, Activities
  Notation:
    -	n_c – number of criteria / criteria list length

Step 2 - basic definitions are set and next step is providing criteria comparision list

Step 3 - Criteria Comparison List

User should assign a criteria rating to each valid criteria pair.
Valid criteria pairs are combinations of pairs from criteria list without considering the order of importance within each pair
Example:
For criteria list = ['Price', 'Performance', 'Climate'], the valid criteria pairs are:
- criterion_1 = "Price", criterion_2 = "Performance"
- criterion_1 = "Price", criterion_2 = "Climate"
- criterion_1 = "Performance", criterion_2 = "Climate"

During the each comparison user should assign a rating come from following criteria rating list to a criteria pair:

['Equally Important', 'Moderately More Important', 'Strongly More Important', 'Very Strongly More Important', 'Extremely More Important', 'Moderately Less Important', 'Strongly Less Important', 'Very Strongly Less Important', 'Extremely Less Important']

The valid criteria comparison statement format:
[criterion_1]  is [criteria_rating]  to/than [criterion_2] , where : criterion_1 and criterion_2 is a valid criteria pair and criteria_rating is value comes from criteria_rating_list
Examples: For criteria_list = ['Price', 'Performance', 'Climate'], sample criteria comparision list is:
- criterion_1 = "Price", criterion_2 = "Performance", criteria_rating = "Equally Important" -> "'Price' is Equally Important to 'Performance'"
- criterion_1 = "Price", criterion_2 = "Climate", criteria_rating = "Moderately More Important" -> "'Price' is Moderately More Important than 'Performance'"
- criterion_1 = "Performance", criterion_2 = "Climate", criteria_rating = "Extremely Less Important" -> "'Price' is Extremely Less Important than 'Performance'"

Notation:
  -	n_cc – number of criteria comparison statements / criteria comparision list length

Step 4 - criteria comparisions are set and and next step is providing preferences statements.

Step 5 - Preferences Dictionary

Preferences dictionary contains preferences list for each criterion in criteria list.

Preferences list contains a list of valid preferences statements for specific criterion.

Valid options pairs are combinations of pairs from options list without considering the order of importance within each pair.
The order between options pairs is important.
Example: For options list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], valid options pairs are:
- option_1 = "Laptop 1", option_2 = "Laptop 2"
- option_1 = "Laptop 1", option_2 = "Laptop 3"
- option_1 = "Laptop 2", option_2 = "Laptop 3"

The valid preference statement format:
[criterion]: [option_1] is [preference_rating] to [option_2], 
where criterion is a value from criteria list, option_1 and option_2 is a valid options pair, preferences_rating is value comes from preference_rating_list:

['Equally Preferred', 'Moderately More Preferred', 'Strongly More Preferred', 'Very Strongly More Preferred', 'Extremely More Preferred', 'Moderately Less Preferred', 'Strongly Less Preferred', 'Very Strongly Less Preferred', 'Extremely Less Preferred']

Preferences dictionary contains criteria in the order as they appear in the criteria list, while the preference statements follow the order of the valid options pairs.
Example: For criteria_list = ['Price', 'Performance', 'Climate'] and options_list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], sample preferences dictionary is:
-	criterion = "Price", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Equally Preferred" -> preference_statement = "'Price': 'Laptop 1' is Equally Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Price': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly Less Preferred" -> preference_statement = "'Price': 'Laptop 2' is Very Strongly Less Preferred than 'Laptop 3'"
-	criterion = "Performance", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Extremely More Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Extremely More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately Less Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Moderately Less Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Performance': 'Laptop 2' is Very Strongly More Preferred than 'Laptop 3'"
-	criterion = "Climate", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Very Strongly More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Extremely Less Preferred" -> preference_statement = "'Climate': 'Laptop 2' is Extremely Less Preferred than 'Laptop 3'"

Notation:
  -	n_oc – for givern criterion , number of preference statements / preferences list length

Step 6 - all definitions are set and the solution will be prepared.


Conversation
============

user: I'm ready to start the decision-making process.
assistant: Great! Let's start by defining your goal. What is the goal you are trying to achieve in an optimal way?
user: Selecting the best laptop
assistant: Now, please provide a list of options you are considering for your laptop selection. What are the different laptop models you have in mind?
user: Gaming Laptop, Ultrabook, 2-in-1 Laptop
assistant: Now, let's define the criteria that will influence your decision in selecting the best laptop. What are the important factors you care about when choosing a laptop?
user: Storage Capacity, Graphics Card, Portability
assistant: Let's move on to the criteria comparison list. Please provide a comparison rating for the following criteria pair: 'Storage Capacity' is compared to 'Graphics Card'. How would you rate their importance?
user: 'Storage Capacity' is Strongly Less Important than 'Graphics Card'
assistant: Let's continue with the criteria comparison list. Please provide a comparison rating for the following criteria pair: 'Storage Capacity' is compared to 'Portability'. How would you rate their importance?
user: Storage Capacity is Extremely Less Important than Graphics Card
assistant: Let's continue with the criteria comparison list. Please provide a comparison rating for the following criteria pair: 'Graphics Card' is compared to 'Portability'. How would you rate their importance?
user: 'Graphics Card' is Strongly Less Important than 'Portability'
assistant: Let's move on to the preferences dictionary. Please provide a preference statement for the criterion 'Storage Capacity' comparing 'Gaming Laptop' and 'Ultrabook'. How would you rate their preference?
user: 'Storage Capacity': 'Gaming Laptop' is Equally Preferred to 'Ultrabook'
assistant: Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Storage Capacity' comparing 'Gaming Laptop' and '2-in-1 Laptop'. How would you rate their preference?
user: 'Storage Capacity': 'Gaming Laptop' is Strongly Less Preferred than '2-in-1 Laptop'
assistant: Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Graphics Card' comparing 'Gaming Laptop' and 'Ultrabook'. How would you rate their preference?
user: 'Graphics Card': 'Gaming Laptop' is Equally Preferred to 'Ultrabook'
assistant: Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Storage Capacity' comparing 'Ultrabook' and '2-in-1 Laptop'. How would you rate their preference?
user: 'Storage Capacity': 'Ultrabook' is Equally Preferred to '2-in-1 Laptop'
assistant: Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Graphics Card' comparing 'Gaming Laptop' and '2-in-1 Laptop'. How would you rate their preference?
user: 'Graphics Card': 'Gaming Laptop' is Equally Preferred to '2-in-1 Laptop'
assistant: Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Graphics Card' comparing 'Ultrabook' and '2-in-1 Laptop'. How would you rate their preference?
user: 'Graphics Card': 'Ultrabook' is Strongly More Preferred than '2-in-1 Laptop'
assistant: Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Portability' comparing 'Gaming Laptop' and 'Ultrabook'. How would you rate their preference?
user: Storage Capacity: Ultrabook is Extremely More Preferred to 2-in-1 Laptop

Process State
=============

---  ---
goal_name:
	Selecting the best laptop
options_list:
	- Gaming Laptop
	- Ultrabook
	- 2-in-1 Laptop
criteria_list:
	- Storage Capacity
	- Graphics Card
	- Portability
criteria_comparison_list:
	- criterion_1: Storage Capacity, criterion_2: Graphics Card, criteria_rating: Strongly Less Important
	- criterion_1: Storage Capacity, criterion_2: Portability, criteria_rating: Extremely Less Important
	- criterion_1: Graphics Card, criterion_2: Portability, criteria_rating: Strongly Less Important
preferences_dict:
	- Storage Capacity:
		-- option_1: Gaming Laptop, option_2: Ultrabook, preference_rating: Equally Preferred
		-- option_1: Gaming Laptop, option_2: 2-in-1 Laptop, preference_rating: Strongly Less Preferred
		-- option_1: Ultrabook, option_2: 2-in-1 Laptop, preference_rating: Equally Preferred
	- Graphics Card:
		-- option_1: Gaming Laptop, option_2: Ultrabook, preference_rating: Equally Preferred
		-- option_1: Gaming Laptop, option_2: 2-in-1 Laptop, preference_rating: Equally Preferred
		-- option_1: Ultrabook, option_2: 2-in-1 Laptop, preference_rating: Strongly More Preferred
	- Portability:
		-- option_1: Gaming Laptop, option_2: Ultrabook, preference_rating: Extremely More Preferred
		-- option_1: Gaming Laptop, option_2: 2-in-1 Laptop, preference_rating: Extremely More Preferred
		-- option_1: Ultrabook, option_2: 2-in-1 Laptop, preference_rating: Extremely More Preferred

Task
====

1. Analyze "Process State" and "Process Technical Explanation" carefully to determine what should be the best next assistant response to user 
   to fill the most critical missing definitions in "Process State" based on "Process Technical Explanation" using guidelines below and set the response as assistant_message. 
   Guidelines:
    - Next assistant response should deal with one definition at time: goal, options list, criteria list, specific criteria comparision statement, or specific preference statement.
    - Next assistant response shouldn't expose any information about content of "Process Technical Explanation".
    - Next assistant response polite and, when it makes sense, encourages user to go on with the process.
    - Next assistant response shouldn't contain any example answers.
    - Next assistant response should enclose options or criteria names with single quotes.
    - Continue to Step 6 in "Process Technical Explanation" if there are no preference statements with preference_rating = 'Missing' in "Process State".
2. If you can conclude from assistant_message that all definitions are set and the solution will be prepared? Yes/No . Set your answer as response_finished.
3. If response_finished is "No": your goal is to provide a list of possible user responses using guidelines below and set these responses to user_examples.
   Guidelines:
    - If assistant_message asks user to provide a goal, generate 3 possible goals to achieve.
    - If assistant_message asks user to provide an options list, generate a list of 3 strings, where each string consisting of 3 possible comma-delimited options. Ensure that each option name is capitalized.
    - If assistant_message asks user to provide a criteria list, generate a list of 3 strings, where each string consisting of 3 possible comma-delimited criteria. Ensure that each criteria name is capitalized.
    - If assistant_message asks user to provide a criteria comparision statement, generate a list of 3 valid criteria comparision statements following their format based on criteria pair mentioned in the assistant_message.
    - If assistant_message asks user to provide a preference statement, generate a list of 3 valid preference statements following their format based on criterion and options pair mentioned in the assistant_message.
    - Each possible user response shouldn't expose any information about content of "Process Technical Explanation".
    - Each possible user response shouldn't contain periods.
    - Each possible user response should enclose options or criteria names with single quotes.
4. If response_finished is "Yes" , set user_examples as empty list.

Task Output: format the output as a JSON object as follows:
```json
  assistant_message: ?
  user_examples: ?
  response_finished: ?
```


In [32]:
from typing import Optional, Literal, List, Dict, Tuple

def get_response_data(prompt : str) -> Dict[str, str | List[str]]:
  import os
  from langchain_openai import ChatOpenAI
  from pydantic import BaseModel, Field
  from typing import List, Literal
  from langchain_core.prompts import ChatPromptTemplate
  from langchain_core.prompts import PromptTemplate
  from langchain_core.output_parsers import StrOutputParser
  from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
  from langchain_core.utils.function_calling import convert_to_openai_function

  model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.1)

  output_parser = StrOutputParser()

  prompt = PromptTemplate(
    template=prompt,
  )

  chain = prompt | model | output_parser

  generation = chain.invoke({'prompt' : prompt})

  import json

  start_string = "```json" ; end_string = "```"
  start_index = generation.find(start_string) + len(start_string)
  end_index = generation.rfind(end_string) - 1

  json_string = generation[start_index:end_index]
  response_data = json.loads(json_string)

  return response_data


In [33]:
response_data = get_response_data(get_response_data_prompt)

In [34]:
import time
start_time = time.time()

try:
  response_data = get_response_data(get_response_data_prompt)
except Exception as e:
  deoptima_logger.error(f"Error in {get_response_data.__name__}: {e}", exc_info=True)
else:
  deoptima_logger.info(f"{get_response_data.__name__} finished: {str(response_data)}")


end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")

deoptima - INFO     : 2024-10-27 00:10:36,054 : get_response_data finished: {'assistant_message': "Let's continue with the preferences dictionary. Please provide a preference statement for the criterion 'Portability' comparing 'Gaming Laptop' and '2-in-1 Laptop'. How would you rate their preference?", 'user_examples': ["'Portability': 'Gaming Laptop' is Equally Preferred to '2-in-1 Laptop'", "'Portability': 'Gaming Laptop' is Strongly More Preferred than '2-in-1 Laptop'", "'Portability': 'Gaming Laptop' is Moderately Less Preferred than '2-in-1 Laptop'"], 'response_finished': 'No'}


Function runtime: 4246.14 msec


## get_validation_data

In [35]:
# # update_entry = {'role': 'user', 'content': 'I think Price is Strongly More Less Important than Climate'}
# # conversation[8] = update_entry
# conversation = conversation[:-10]

In [36]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

from deoptima.utils import conversation_to_string
from deoptima.utils import process_state_to_string

In [37]:
import os
import yaml

with open(os.path.join(SYS_SRC_DIR, 'generation_data.yaml'), 'r') as file:
    generation_data = yaml.safe_load(file)

In [38]:
criteria_rating_list = generation_data['infos']['criteria_rating_list']
preference_rating_list = generation_data['infos']['preference_rating_list']
process_technical_explanation = generation_data['infos']['process_technical_explanation']
process_technical_explanation = process_technical_explanation.format(
    criteria_rating_list=criteria_rating_list,
    preference_rating_list=preference_rating_list)
get_validation_data_prompt = generation_data['prompts']['get_validation_data']['prompt']
get_validation_data_prompt = get_validation_data_prompt.format(
    process_technical_explanation=process_technical_explanation,
    process_state=process_state_to_string(process_state))

In [39]:
from IPython.display import Markdown
display(Markdown(f"{get_validation_data_prompt}"))

Process Technical Explanation
=============================

Step 1 - Basic Definitions: User should provide following list of definitions during conversation with Deoptima:

-	Step 1.1 Goal the user trying to achieve in optimal way
  Examples:
    -	selecting the best laptop
    -	selecting the best destination for your vacation
-	Step 1.2 Options list: options (or alternatives) are the different choices or possibilities that the user can select from to achieve goal
  Examples:
    -	Laptop model 1, Laptop model 2, Laptop model 3
    -	Vacation Destination 1, Vacation Destination 2, Vacation Destination 3
  Notation:
    -	n_o – number of options / options list length
-	Step 1.3 Criteria list: Criteria are the important factors that will influence your decision. Think of them as the things user care most about when choosing the best option.
  Examples:
    -	Price, Performance, Battery Life
    -	Price, Climate, Activities
  Notation:
    -	n_c – number of criteria / criteria list length

Step 2 - basic definitions are set and next step is providing criteria comparision list

Step 3 - Criteria Comparison List

User should assign a criteria rating to each valid criteria pair.
Valid criteria pairs are combinations of pairs from criteria list without considering the order of importance within each pair
Example:
For criteria list = ['Price', 'Performance', 'Climate'], the valid criteria pairs are:
- criterion_1 = "Price", criterion_2 = "Performance"
- criterion_1 = "Price", criterion_2 = "Climate"
- criterion_1 = "Performance", criterion_2 = "Climate"

During the each comparison user should assign a rating come from following criteria rating list to a criteria pair:

['Equally Important', 'Moderately More Important', 'Strongly More Important', 'Very Strongly More Important', 'Extremely More Important', 'Moderately Less Important', 'Strongly Less Important', 'Very Strongly Less Important', 'Extremely Less Important']

The valid criteria comparison statement format:
[criterion_1]  is [criteria_rating]  to/than [criterion_2] , where : criterion_1 and criterion_2 is a valid criteria pair and criteria_rating is value comes from criteria_rating_list
Examples: For criteria_list = ['Price', 'Performance', 'Climate'], sample criteria comparision list is:
- criterion_1 = "Price", criterion_2 = "Performance", criteria_rating = "Equally Important" -> "'Price' is Equally Important to 'Performance'"
- criterion_1 = "Price", criterion_2 = "Climate", criteria_rating = "Moderately More Important" -> "'Price' is Moderately More Important than 'Performance'"
- criterion_1 = "Performance", criterion_2 = "Climate", criteria_rating = "Extremely Less Important" -> "'Price' is Extremely Less Important than 'Performance'"

Notation:
  -	n_cc – number of criteria comparison statements / criteria comparision list length

Step 4 - criteria comparisions are set and and next step is providing preferences statements.

Step 5 - Preferences Dictionary

Preferences dictionary contains preferences list for each criterion in criteria list.

Preferences list contains a list of valid preferences statements for specific criterion.

Valid options pairs are combinations of pairs from options list without considering the order of importance within each pair.
The order between options pairs is important.
Example: For options list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], valid options pairs are:
- option_1 = "Laptop 1", option_2 = "Laptop 2"
- option_1 = "Laptop 1", option_2 = "Laptop 3"
- option_1 = "Laptop 2", option_2 = "Laptop 3"

The valid preference statement format:
[criterion]: [option_1] is [preference_rating] to [option_2], 
where criterion is a value from criteria list, option_1 and option_2 is a valid options pair, preferences_rating is value comes from preference_rating_list:

['Equally Preferred', 'Moderately More Preferred', 'Strongly More Preferred', 'Very Strongly More Preferred', 'Extremely More Preferred', 'Moderately Less Preferred', 'Strongly Less Preferred', 'Very Strongly Less Preferred', 'Extremely Less Preferred']

Preferences dictionary contains criteria in the order as they appear in the criteria list, while the preference statements follow the order of the valid options pairs.
Example: For criteria_list = ['Price', 'Performance', 'Climate'] and options_list = ['Laptop 1', 'Laptop 2', 'Laptop 3'], sample preferences dictionary is:
-	criterion = "Price", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Equally Preferred" -> preference_statement = "'Price': 'Laptop 1' is Equally Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Price': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly Less Preferred" -> preference_statement = "'Price': 'Laptop 2' is Very Strongly Less Preferred than 'Laptop 3'"
-	criterion = "Performance", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Extremely More Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Extremely More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately Less Preferred" -> preference_statement = "'Performance': 'Laptop 1' is Moderately Less Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Performance': 'Laptop 2' is Very Strongly More Preferred than 'Laptop 3'"
-	criterion = "Climate", preferences list contains following valid preference statements:
  --	option_1 = "Laptop 1", option_2 = "Laptop 2", preferences_rating = "Very Strongly More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Very Strongly More Preferred to 'Laptop 2'"
  --	option_1 = "Laptop 1", option_2 = "Laptop 3", preferences_rating = "Moderately More Preferred" -> preference_statement = "'Climate': 'Laptop 1' is Moderately More Preferred than 'Laptop 3'"
  --	option_1 = "Laptop 2", option_2 = "Laptop 3", preferences_rating = "Extremely Less Preferred" -> preference_statement = "'Climate': 'Laptop 2' is Extremely Less Preferred than 'Laptop 3'"

Notation:
  -	n_oc – for givern criterion , number of preference statements / preferences list length

Step 6 - all definitions are set and the solution will be prepared.


Process State
=============

---  ---
goal_name:
	Selecting the best laptop
options_list:
	- MacBook Pro
	- Dell XPS
	- Lenovo ThinkPad
criteria_list:
	- Storage Capacity
	- Portability
	- Customer Support
criteria_comparison_list:
	- criterion_1: Storage Capacity, criterion_2: Portability, criteria_rating: Strongly Less Important
	- criterion_1: Storage Capacity, criterion_2: Customer Support, criteria_rating: Very Strongly Less Important
	- criterion_1: Portability, criterion_2: Customer Support, criteria_rating: Moderately More Important
preferences_dict:
	- Storage Capacity:
		-- option_1: MacBook Pro, option_2: Dell XPS, preference_rating: Moderately Less Preferred
		-- option_1: MacBook Pro, option_2: Lenovo ThinkPad, preference_rating: Strongly More Preferred
		-- option_1: Dell XPS, option_2: Lenovo ThinkPad, preference_rating: Strongly Less Preferred
	- Portability:
		-- option_1: MacBook Pro, option_2: Dell XPS, preference_rating: Missing
		-- option_1: MacBook Pro, option_2: Lenovo ThinkPad, preference_rating: Missing
		-- option_1: Dell XPS, option_2: Lenovo ThinkPad, preference_rating: Missing
	- Customer Support:
		-- option_1: MacBook Pro, option_2: Dell XPS, preference_rating: Missing
		-- option_1: MacBook Pro, option_2: Lenovo ThinkPad, preference_rating: Missing
		-- option_1: Dell XPS, option_2: Lenovo ThinkPad, preference_rating: Missing

Task
==============
Perform the following validity checks based on the "Process Technical Explanation" and "Process State".

validity_check_1:
  name: Criteria comparison list with valid criteria pairs
  condition: Each criteria pair in criteria_comparison_list is a valid criteria pair
  status: Set "Pass" if condition holds, "Fail" - if condition doesn't hold, "Skipped" - if the check wasn't reached.
validity_check_2:
  name: Criteria comparison list with valid criteria ratings
  condition: Each criteria pair in criteria_comparison_list has a criteria_rating from criteria_rating_list.
  status: Set "Pass" if condition holds, "Fail" - if condition doesn't hold, "Skipped" - if the check wasn't reached.
validity_check_3:
  name: Preferences dictionary with valid criteria keys
  condition: Each criteria in preferences_dictionary is in criteria_list
  status: Set "Pass" if condition holds, "Fail" - if condition doesn't hold, "Skipped" - if the check wasn't reached.

Stop as soon as you encounter the first failure:
1.1 Analyze the Process State and verify if the condition holds true for each validity check.
    Provide a detailed step-by-step explanation to verify the condition. Ensure that only this condition is assessed without attempting to validate any other aspects. Set the explanation as sbsc_explanation.
1.2 Set the status field of failed validity check to "Fail" and "Skipped" to rest of vaidity checks and go to "All validity checks passed" section.
1.3 If a validity check fails:
  1.3.1 Generate a polite and consise message describing the problem to user, referring sbsc_explanation. Set the message as assistant_message.
  1.3.2 Generate empty list. Set it to user_examples.
All validity checks passed:
1.1 Generate a message informing user about all definitions are set and the solution will be prepared. Set the message as assistant_message.
1.2 Generate empty list. Set it to user_examples.

Task Output: After all checks are completed (or a failure is encountered), format the output as a JSON object as follows
1. validity_checks:
  - name
  - sbsc_explanation
  - status
2. response_data:
  - assistant_message
  - user_examples


In [40]:
from typing import List, Dict, Optional, Tuple

def get_validation_data(prompt : str):
  import os
  from pydantic import BaseModel, Field
  from langchain_core.utils.function_calling import convert_to_openai_function
  from langchain_openai import ChatOpenAI
  from langchain_core.output_parsers import StrOutputParser
  from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
  from langchain_core.prompts import PromptTemplate
  from langchain_core.prompts import ChatPromptTemplate

  output_parser = StrOutputParser()

  model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

  prompt = PromptTemplate(
    template=prompt,
    # input_variables=["missing_criteria_comparison_sample_statements"],
  )

  chain = prompt | model | output_parser

  generation = chain.invoke({'prompt' : prompt})

  # print(generation)

  import json

  start_string = "```json" ; end_string = "```"
  start_index = generation.find(start_string) + len(start_string)
  end_index = generation.rfind(end_string) - 1

  json_string = generation[start_index:end_index]
  response_data = json.loads(json_string)

  return response_data

In [41]:
validation_data = get_validation_data(get_validation_data_prompt)

In [42]:
import time
start_time = time.time()

try:
  validation_data = get_validation_data(get_validation_data_prompt)
except Exception as e:
  deoptima_logger.error(f"Error in {get_validation_data.__name__}: {e}", exc_info=True)
else:
  deoptima_logger.info(f"{get_validation_data.__name__} finished: {str(validation_data)}")

end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")

deoptima - INFO     : 2024-10-26 23:44:44,375 : get_validation_data finished: {'validity_checks': [{'name': 'Criteria comparison list with valid criteria pairs', 'sbsc_explanation': "To verify if each criteria pair in the criteria_comparison_list is valid, we need to check if the pairs are combinations of the criteria in the criteria_list without considering the order of importance. The criteria_list contains: ['Storage Capacity', 'Portability', 'Customer Support']. The valid pairs are: ('Storage Capacity', 'Portability'), ('Storage Capacity', 'Customer Support'), and ('Portability', 'Customer Support'). The criteria_comparison_list contains the following pairs: ('Storage Capacity', 'Portability'), ('Storage Capacity', 'Customer Support'), and ('Portability', 'Customer Support'). Since all pairs in the criteria_comparison_list are valid combinations from the criteria_list, this check passes.", 'status': 'Pass'}, {'name': 'Criteria comparison list with valid criteria ratings', 'sbsc_exp

Function runtime: 4858.05 msec


In [43]:
print(dictionary_to_string(response_data,'response_data'))

--- response_data ---
assistant_message:
	Now, please provide a preference statement for the pair of options: 'Dell XPS' and 'Lenovo ThinkPad' for the criterion
	'Portability'.
user_examples:
	- Portability: 'Dell XPS' is Equally Preferred to 'Lenovo ThinkPad'
	- Portability: 'Dell XPS' is Moderately More Preferred than 'Lenovo ThinkPad'
	- Portability: 'Dell XPS' is Strongly Less Preferred than 'Lenovo ThinkPad'
response_finished:
	No


## get_validation_data_func

In [44]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

from deoptima.utils import conversation_to_string
from deoptima.utils import process_state_to_string

In [45]:
import os
import yaml

with open(os.path.join(SYS_SRC_DIR, 'generation_data.yaml'), 'r') as file:
    generation_data = yaml.safe_load(file)

In [46]:
criteria_rating_list = generation_data['infos']['criteria_rating_list']
preference_rating_list = generation_data['infos']['preference_rating_list']

In [47]:
def get_validation_data_func(process_state: Dict, criteria_rating_list: List, preference_rating_list: List) -> Dict:

  import os
  import re
  import random
  from deoptima.utils import dictionary_to_string

  response = {}; response['validity_checks'] = {}; response['response_data'] = {}

  options_list = process_state['options_list']
  criteria_list = process_state['criteria_list']
  criteria_comparision_list = process_state['criteria_comparison_list']
  preferences_dict = process_state['preferences_dict']

  validity_checks = [
    {
      'name' : "Criteria comparison list with valid criteria pairs",
      'status': "Not Started"
    },
    {
      'name' : "Criteria comparison list with valid criteria ratings",
      'status': "Not Started"
    },
    {
      'name' : "Preferences dictionary with valid criteria keys",
      'status': "Not Started"
    },
    {
      'name' : "Preferences dictionary with valid options pairs",
      'status': "Not Started"
    },
    {
      'name' : "Preferences dictionary with valid preference ratings", # Under development
      'status': "Not Started"
    },
  ]

  ### validity_check_0 - "Criteria comparison list with valid criteria pairs"
  import itertools
  partial_status = []
  sbsc_explanation = {}
  valid_criteria_pairs = list(itertools.combinations(criteria_list, 2))
  sbsc_explanation['valid_criteria_pairs'] = valid_criteria_pairs
  sbsc_explanation['criteria_comparision_list'] = criteria_comparision_list
  for c in criteria_comparision_list:
    criteria_pair = (c['criterion_1'], c['criterion_2'])
    if criteria_pair in valid_criteria_pairs:
      partial_status.append('Pass')
    else:
      partial_status.append('Fail')
  sbsc_explanation['partial_status'] = partial_status
  validity_checks[0]['sbsc_explanation'] = sbsc_explanation
  validity_checks[0]['status'] = "Pass" if all(i == 'Pass' for i in partial_status) else "Fail"

  ### validity_check_1 - "Criteria comparison list with valid criteria ratings"
  partial_status = []
  sbsc_explanation = {}
  sbsc_explanation['criteria_rating_list'] = criteria_rating_list
  sbsc_explanation['criteria_comparision_list'] = criteria_comparision_list
  for c in criteria_comparision_list:
    criteria_rating = c['criteria_rating']
    if criteria_rating in criteria_rating_list:
      partial_status.append('Pass')
    else:
      partial_status.append('Fail')
  sbsc_explanation['partial_status'] = partial_status
  validity_checks[1]['sbsc_explanation'] = sbsc_explanation
  validity_checks[1]['status'] = "Pass" if all(i == 'Pass' for i in partial_status) else "Fail"

  ### validity_check_2 - "Preferences dictionary with valid criteria keys" - List Version
  partial_status = []
  sbsc_explanation = {}
  sbsc_explanation['criteria_list'] = criteria_list
  preferences_criteria_list = list(preferences_dict.keys()) # preferences_dict is dict
  sbsc_explanation['preferences_criteria_list'] = preferences_criteria_list
  for c in preferences_criteria_list:
    if c in criteria_list:
      partial_status.append('Pass')
    else:
      partial_status.append('Fail')
  sbsc_explanation['partial_status'] = partial_status
  validity_checks[2]['sbsc_explanation'] = sbsc_explanation
  validity_checks[2]['status'] = "Pass" if all(i == 'Pass' for i in partial_status) else "Fail"

  ### validity_check_3 - "Preferences dictionary with valid options pairs"
  import itertools
  partial_status = []
  sbsc_explanation = {}
  valid_preference_triplets = [(criterion, option_1, option_2)
                               for criterion in criteria_list
                               for option_1, option_2 in itertools.combinations(options_list, 2)]
  sbsc_explanation['valid_preference_triplets'] = valid_preference_triplets
  # preferences_dict is dict
  actual_preferences_triplets = [(criterion, preference['option_1'], preference['option_2'])
                                for criterion, preferences in preferences_dict.items()
                                for preference in preferences]
  sbsc_explanation['actual_preferences_triplets'] = actual_preferences_triplets
  for c in actual_preferences_triplets:
    if c in valid_preference_triplets:
      partial_status.append('Pass')
    else:
      partial_status.append('Fail')
  sbsc_explanation['partial_status'] = partial_status
  validity_checks[3]['sbsc_explanation'] = sbsc_explanation
  validity_checks[3]['status'] = "Pass" if all(i == 'Pass' for i in partial_status) else "Fail"

  ### validity_check_4 - "Preferences dictionary with valid preference ratings"
  partial_status = []
  sbsc_explanation = {}
  sbsc_explanation['preference_rating_list'] = preference_rating_list
  sbsc_explanation['preferences_dict'] = preferences_dict
  # preferences_dict is dict
  for c in preferences_dict:
    for ps in preferences_dict[c]:
      preference_rating = ps['preference_rating']
      if preference_rating in preference_rating_list:
        partial_status.append('Pass')
      else:
        partial_status.append('Fail')
  sbsc_explanation['partial_status'] = partial_status
  validity_checks[4]['sbsc_explanation'] = sbsc_explanation
  validity_checks[4]['status'] = "Pass" if all(i == 'Pass' for i in partial_status) else "Fail"


  assistant_examples = [
    "Great! All the inputs are in place. I’ll now analyze the information and present the best solution.",
    "Everything is set! I’ll now process the data and deliver the most optimal option for your definitions.",
    "All definitions have been confirmed. Let me work through the details and offer you the best possible solution."
  ]

  assistant_message = random.choice(assistant_examples)

  response_data = {
    'assistant_message': assistant_message,
    'user_examples': []
  }

  response['validity_checks'] = validity_checks; response['response_data'] = response_data

  return response


# generate_model_response

## Setup

In [48]:
scenario_code = "S0"
# scenario_code = "S1"

In [49]:
import os

conversation_input_folder = os.path.join(SYS_TESTING_DIR, "input", scenario_code)
conversation_files = [f for f in os.listdir(conversation_input_folder) if os.path.isfile(os.path.join(conversation_input_folder, f))]

conversation_files

['abc.json']

In [50]:
from deoptima.utils import load_from_json

conversation_idx = 0
conversation = load_from_json(os.path.join(conversation_input_folder, conversation_files[conversation_idx]))
# conversation = conversation[:-9]

In [51]:
from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

In [52]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

import yaml
from deoptima.utils import dictionary_to_string
from deoptima.utils import conversation_to_string
from deoptima.utils import process_state_to_string
from deoptima.utils import generate_unique_hash

class Generation_Data:
    def __init__(self, data_path):
        self.data_path = data_path
        self.generation_data = self.load_yaml_data('generation_data.yaml')
        self.criteria_rating_list = self.generation_data['infos']['criteria_rating_list']
        self.preference_rating_list = self.generation_data['infos']['preference_rating_list']
        self.process_technical_explanation = self.format_technical_explanation()

    def load_yaml_data(self, filename):
        """Loads YAML data from the specified file."""
        import os

        file_path = os.path.join(self.data_path, filename)
        try:
            with open(file_path, 'r') as file:
                return yaml.safe_load(file)
        except FileNotFoundError:
          error_message = f"File {filename} not found at {self.data_path}"
          deoptima_logger.error(error_message) ;  raise Exception(error_message)
        except yaml.YAMLError as e:
          error_message = f"Error parsing YAML file: {e}"
          deoptima_logger.error(error_message) ;  raise Exception(error_message)

    def get_criteria_rating_list(self):
      return self.criteria_rating_list

    def get_preference_rating_list(self):
      return self.preference_rating_list

    def format_technical_explanation(self):
        """Formats the process technical explanation."""
        process_technical_explanation = self.generation_data['infos']['process_technical_explanation']
        return process_technical_explanation.format(criteria_rating_list = self.criteria_rating_list,
                                                    preference_rating_list = self.preference_rating_list)

    def get_process_state_prompt(self):
        """Property for retrieving the process state prompt."""
        prompt_template = self.generation_data['prompts']['get_process_state']['prompt']
        return prompt_template.format(
              process_technical_explanation=self.process_technical_explanation
          )

    def get_response_data_prompt(self, conversation, process_state):
        """Method for retrieving the response data prompt."""
        prompt_template = self.generation_data['prompts']['get_response_data']['prompt']
        return prompt_template.format(
              process_technical_explanation=self.process_technical_explanation,
              conversation=conversation_to_string(conversation),
              process_state=process_state_to_string(process_state)
          )

    def get_validation_data_prompt(self, conversation, process_state):
        """Method for retrieving the validation data prompt."""
        prompt_template = self.generation_data['prompts']['get_validation_data']['prompt']
        return prompt_template.format(
              process_technical_explanation=self.process_technical_explanation,
              conversation=conversation_to_string(conversation),
              process_state=process_state_to_string(process_state)
         )

generation_data = Generation_Data(SYS_SRC_DIR)

def generate_model_response(conversation: List[Dict[str, str]]):

  ### process_state
  process_state = None
  try:
    get_process_state_prompt = generation_data.get_process_state_prompt()
    process_state = get_process_state(get_process_state_prompt, conversation_to_string(conversation))
  except Exception as e:
    deoptima_logger.error(f"Error in 'get_process_state': {e}", exc_info=True)
  else:
    deoptima_logger.info(f"{'get_process_state - get_response_data'} finished: \n{process_state_to_string(process_state,'process_state')}")
  ### response_data
  response_data = None
  try:
    get_response_data_prompt = generation_data.get_response_data_prompt(conversation, process_state)
    response_data = get_response_data(get_response_data_prompt)
  except Exception as e:
    deoptima_logger.error(f"Error in 'get_response_data': {e}", exc_info=True)
  else:
    deoptima_logger.info(f"{'get_response_data'} finished: \n {dictionary_to_string(response_data,'response_data')}")

  if (response_data['response_finished'] == "No"):
    return response_data['response_finished'], False, response_data['assistant_message'], response_data['user_examples']
  ### process_state
  process_state = None
  try:
    get_process_state_prompt = generation_data.get_process_state_prompt()
    process_state = get_process_state(get_process_state_prompt, conversation_to_string(conversation))
  except Exception as e:
    deoptima_logger.error(f"Error in 'get_process_state': {e}", exc_info=True)
  else:
    deoptima_logger.info(f"{'get_process_state - get_validation_data'} finished: \n{process_state_to_string(process_state,'process_state')}")
  # ### validation_data
  # try:
  #   get_validation_data_prompt = generation_data.get_validation_data_prompt(conversation, process_state)
  #   validation_data = get_validation_data(get_validation_data_prompt)
  #   all_validity_checks_passed = all(validity_check['status'] == "Pass" for validity_check in validation_data['validity_checks'])
  # except Exception as e:
  #   deoptima_logger.error(f"Error in 'get_validation_data': {e}", exc_info=True)
  # else:
  #   info_message = f"{'get_validation_data'} finished: \n{dictionary_to_string(validation_data,'validation_data')}\n all_validity_checks_passed : {all_validity_checks_passed}"
  #   deoptima_logger.info(info_message)
  ### validation_data_func
  try:
    criteria_rating_list = generation_data.get_criteria_rating_list()
    preference_rating_list = generation_data.get_preference_rating_list()
    validation_data = get_validation_data_func(process_state, criteria_rating_list, preference_rating_list)
    all_validity_checks_passed = all(validity_check['status'] == "Pass" for validity_check in validation_data['validity_checks'])
  except Exception as e:
    deoptima_logger.error(f"Error in 'get_validation_data': {e}", exc_info=True)
  else:
    info_message = f"{'get_validation_data'} finished: \n{dictionary_to_string(validation_data,'validation_data')}\n all_validity_checks_passed : {all_validity_checks_passed}"
    deoptima_logger.info(info_message)

  return response_data['response_finished'], all_validity_checks_passed, validation_data['response_data']['assistant_message'], validation_data['response_data']['user_examples']


## Sample Run

In [53]:
# assistant_message,  user_examples = generate_model_response(conversation)

In [54]:
import time
start_time = time.time()

deoptima_logger.info(f"{generate_model_response.__name__} started.")
try:
  response_finished, all_validity_checks_passed, assistant_message,  user_examples = generate_model_response(conversation)
except Exception as e:
  deoptima_logger.error(f"Error in {generate_model_response.__name__}: {e}", exc_info=True)
else:
  deoptima_logger.info(f"{generate_model_response.__name__} finished.")

conversation.append({"role": "assistant", "content": assistant_message})
import random
conversation.append({"role": "user", "content": random.choice(user_examples)})

end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")

deoptima - INFO     : 2024-10-26 23:44:44,874 : generate_model_response started.
deoptima - INFO     : 2024-10-26 23:44:45,731 : get_process_state - get_response_data finished: 
--- process_state ---
goal_name:
	
options_list:
criteria_list:
criteria_comparison_list:
preferences_dict:
deoptima - INFO     : 2024-10-26 23:44:47,057 : get_response_data finished: 
 --- response_data ---
assistant_message:
	Great! Let's start by defining your goal. What is the goal you are trying to achieve in an optimal way?
user_examples:
	- Selecting the best laptop
	- Choosing the ideal vacation destination
	- Finding the most suitable smartphone
response_finished:
	No
deoptima - INFO     : 2024-10-26 23:44:47,059 : generate_model_response finished.


Function runtime: 2186.80 msec


In [55]:
from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

# Run Scenario

In [56]:
# from deoptima.utils import set_logging_level
# set_logging_level(deoptima_logger, 'DEBUG')

In [57]:
conversation_input_folder = os.path.join(SYS_TESTING_DIR, "input")

# conversation_file = "start_process_1.json"
# conversation_file = "process_1.json"
# conversation_file = "process_2.json"
# conversation_file = "process_3.json"
# conversation_file = "process_5.json"
# conversation_file = "assistant_stop_2.json"
# conversation_file = "process_test_options.json"
conversation_file = "recent_1.json"

conversation = load_from_json(os.path.join(conversation_input_folder, conversation_file))

from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

In [58]:
from deoptima.utils import save_to_json

import time
start_time = time.time()

stop_criterion = False

n_version = 1
n_runs = 1
run_cnt = 1 ; run_limit = 18

for run_id in range(n_runs):
  while not(stop_criterion):
    deoptima_logger.info(f"generate_model_response started.")
    try:
      response_finished, all_validity_checks_passed, assistant_message,  user_examples = generate_model_response(conversation)
    except Exception as e:
      deoptima_logger.error(f"Error in generate_model_response: {e}", exc_info=True)
      raise
    else:
      conversation.append({"role": "assistant", "content": assistant_message})
      if not all_validity_checks_passed:
        import random
        user_message = random.choice(user_examples)
        conversation.append({"role": "user", "content": user_message})

    deoptima_logger.info(f"\n******\n user_message: {user_message}\n assistant_message: {assistant_message}\n user_examples: {user_examples}")

    deoptima_logger.info(f"generate_model_response finished.")
    run_cnt += 1
    stop_criterion = (all_validity_checks_passed) or run_cnt > run_limit

  conversation_output_folder = os.path.join(SYS_TESTING_DIR, "output", scenario_code)
  save_to_json(conversation, os.path.join(conversation_output_folder,
                                          f"{os.path.splitext(conversation_files[conversation_idx])[0]}_{n_version}_{run_id}.json"))

end_time = time.time()
runtime_ms = (end_time - start_time) * 1000
print(f"Function runtime: {runtime_ms:.2f} msec")


deoptima - INFO     : 2024-10-26 23:44:48,009 : generate_model_response started.
deoptima - INFO     : 2024-10-26 23:44:52,749 : get_process_state - get_response_data finished: 
--- process_state ---
goal_name:
	Selecting the best laptop
options_list:
	- Gaming Laptop
	- Ultrabook
	- 2-in-1 Laptop
criteria_list:
	- Storage Capacity
	- Graphics Card
	- Portability
criteria_comparison_list:
	- criterion_1: Storage Capacity, criterion_2: Graphics Card, criteria_rating: Strongly Less Important
	- criterion_1: Storage Capacity, criterion_2: Portability, criteria_rating: Extremely Less Important
	- criterion_1: Graphics Card, criterion_2: Portability, criteria_rating: Strongly Less Important
preferences_dict:
	- Storage Capacity:
		-- option_1: Gaming Laptop, option_2: Ultrabook, preference_rating: Equally Preferred
		-- option_1: Gaming Laptop, option_2: 2-in-1 Laptop, preference_rating: Strongly Less Preferred
		-- option_1: Ultrabook, option_2: 2-in-1 Laptop, preference_rating: Equally Pr

Function runtime: 23836.71 msec


In [60]:
from IPython.display import display, HTML
from deoptima.utils import format_conversation

html_content = format_conversation(conversation)
display(HTML(html_content))

# generation_data.yaml

## Criteria only

In [ ]:
generation_data = """
infos:
  criteria_rating_list:
  - Equally Important
  - Moderately More Important
  - Strongly More Important
  - Very Strongly More Important
  - Extremely More Important
  - Moderately Less Important
  - Strongly Less Important
  - Very Strongly Less Important
  - Extremely Less Important
  options_rating_list:
  - Equally Preferred
  - Moderately More Preferred
  - Strongly More Preferred
  - Very Strongly More Preferred
  - Extremely More Preferred
  - Moderately Less Preferred
  - Strongly Less Preferred
  - Very Strongly Less Preferred
  - Extremely Less Preferred

  process_technical_explanation: |
    Step 1 - Basic Definitions: User should provide following list of definitions during conversation with Deoptima:
    •	Step 1.1 Goal the user trying to achieve in optimal way
      Examples:
        o	selecting the best laptop
        o	selecting the best destination for your vacation
    •	Step 1.2 Options list: options (or alternatives) are the different choices or possibilities that the user can select from to achieve goal
      Examples:
      o	Laptop model 1, Laptop model 2, Laptop model 3
      o	Vacation Destination 1, Vacation Destination 2, Vacation Destination 3
      Notation:
        o	n_o – number of options / options list length
    •	Step 1.3 Criteria list: Criteria are the important factors that will influence your decision. Think of them as the things user care most about when choosing the best option.

      Examples:
        o	Price, Performance, Battery Life
        o	Price, Climate, Activities

      Notation:
        o	n_c – number of criteria / criteria list length

    Step 2 - Criteria Comparison: User should compare each unique, unordered pair of criteria defined on criteria list.

    During the each comparison user should assign a rating come from following criteria rating list to a pair:

    {criteria_rating_list}

    The valid criteria comparison statement that user should provide have format:
    [criterion_1]  is [criteria_rating]  to/than [criterion_2] , where : criterion_1 and criterion_2 are values from criteria list and criteria_rating is value come from criteria_rating_list

    Examples:
    •	For: criterion_1 = “Price”, criterion_2 = “Performance” , sample valid criteria comparison statements are :
      o	“Price is Equally Important to Performance”
      o	“Price is Moderately More Important than Performance” is same as “Performance is Moderately Less Important than Price”
    •	For: criterion 1 = “Price”, criterion 2 = “Climate” , sample valid criteria comparison statements are :
      o	“Price is Equally Important than Climate”
      o	“Price is Very Strongly More Important than Climate” is same as "Climate is Very Strongly Less Important than Price"

    Criteria comparison list defined as a list of criteria comparison statements.
    Criteria comparison list should contain exactly nc*(nc-1)/2 criteria comparison statements.

    Notation:
      o	n_cc – number of criteria comparison statements / criteria comparision list length

    Step 3 - all definitions are set and the solution will be prepared

prompts:
  get_response_data:
    prompt: |
      Process Technical Explanation
      =============================

      {process_technical_explanation}

      Conversation
      ============

      {conversation}

      Process State
      =============

      {process_state}

      Task
      ====

      1. Analyze the Conversation and Process State. Follow all steps in "Process Technical Explanation" carefully to determine what should be the best next assistant response to user.
         Next assistant response shouldn't contain any example answers.
         Set the response as assistant_message.
      2. If you can conclude from assistant_message that all definitions are set and the solution will be prepared? Yes/No . Set your answer as response_finished.
      3. Set user_examples following rules :
         3.1 If response_finished is "No" , given the Conversation and assistant_message, generate 3 possible user responses based on Process State to fill the most critical missing definitions.
         If required user response is a list - generate a comma delimited string.
         Possible use response shouldn't contain periods.
         Set these responses to user_examples.
         3.2 If response_finished is "Yes" , set user_examples as empty list.

      Task Output: format the output as a JSON object as follows
        assistant_message: ?
        user_examples: ?
        response_finished: ?

  get_process_state:
    prompt: |
      Process Technical Explanation
      =============================

      {process_technical_explanation}

      Task
      ============

      A conversation between Deoptima assistant and User. Extract from it process state components : Goal , Options , Criteria , and Criteria comparison statements
      During the conversation each process state component can be updated.
      You should to determine goal, options list, criteria list and their last value stated by user or assistant.
      You should to determine each pair criteria and its last criteria comparison statement stated by user or assistant.

  get_validation_data:
    prompt: |
      Process Technical Explanation
      =============================

      {process_technical_explanation}

      Process State
      =============

      {process_state}

      Task
      ==============
      Perform the following validity checks based on the Process Technical Explanation and Process State.
      Don't expose any technical information from Process Technical Explanation or validity checks.

      validity_check_1:
        name: Criteria comparison statements validity
        condition: Process state contains only valid criteria comparison statements as defined in Step 2
        solution:
          - Find invalid criteria comparison statements to propose user to drop them
        status: Set "Pass" if condition holds, "Fail" - if condition doesn't hold, "Skipped" - if the check wasn't reached.
      validity_check_2:
        name: Number of criteria comparison statements
        condition: n_cc == nc*(nc-1)/2
        solution:
          - Create missing criteria comparison statements and propose user to add them
          - Check a list of criteria comparison statements to find statements that contradict and propose user to drop them
          - Check a list of criteria comparison statements to find statements that redundant and propose user to drop them
        status: Set "Pass" if condition holds, "Fail" - if condition doesn't hold, "Skipped" - if the check wasn't reached.

      Stop as soon as you encounter the first failure:
      1.1 Analyze the Process State and verify if the condition holds true for each validity check.
          Provide a detailed step-by-step explanation of the logic used to verify the condition and to formulate possible solutions. Set the explanation as sbsc_explanation.
      1.2 Set the status field of failed validity check to "Fail" and "Skipped" to rest of vaidity checks and go to "All validity checks passed" section.
      1.3 If a validity check fails:
        1.3.1 Provide a detailed step-by-step explanation of the logic to resolve the problems based on validity check solution. Set the explanation as sbss_explanation.
        1.3.2 Generate a polite and consise message describing the problem to user, referring sbsc_explanation. Set the message as assistant_message.
        1.3.3 Based on the sbss_explanation, generate a sample of up to 3 of the most relevant examples for user responses to instruct the assistant to resolve the problems. Set these options as initial_user_examples.
        1.3.4 Refine initial_user_examples and leave most specific responses. Set these options as user_examples.
      All validity checks passed:
      1.1 Generate a message about all definitions are set and the solution will be prepared. Set the message as assistant_message.
      1.2 Generate empty list. Set it to user_examples.


      Task Output: After all checks are completed (or a failure is encountered), format the output as a JSON object as follows
      1. For each executed validity check fill data as follows :
         1. validity check name
         2. sbsc_explanation
         3. sbss_explanation
         4. validity check status
      2. response_data:
           assistant_message: ?
           user_examples: ?

"""

In [ ]:
# Write the string to a text file
import os
with open(os.path.join(SYS_PROJECT_DIR, "deoptima", "generation_data.yaml"), 'w') as file:
    file.write(generation_data)

8446

## Criteria and Options

# Sleep

In [ ]:
import time
time.sleep(30000)

KeyboardInterrupt: 